# Assignment 6
## Project 1
The first project you can choose is a continuation of the InterPROscan dataset you worked on for Assignment 5. You now know the structure and type of data that is involved. What I would ike you to do is to develop scikit-learn machine learning models to predict the function of the proteins in the dataset. I'm not going to mandate any particular model, but you should use your knowledge from DS3 to determine good candidate methods and try a couple, and report on yourGiven the size of the data, you should train your machine models in parallel! Dask is an obvious candidate for this, but you can also use any of the other frameworks we discussed in the course.

The "function" of the protein which is the "class" your model should predict is defined as the InterPRO number which covers:

1. more than 90% of the protein's sequence
2. Covers the largest length of the sequence

The "features" that you use to predict that class are the other, smaller, InterPRO annotations. Take care to remove "noise" from the dataset, ie. lines from the TSV file which don't contain InterPRO numbers, proteins which don't have a large feature (according to the criteria above) etc. Be sure to note in your report how you processed the data and why you made the choices you did!

Your script should, given one or more of these TSV files with InterPROscan annotations, produce models (Pickle them to save them!) which predict protein function, and a file of training-data on which you trained them.


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Data preperation

In [12]:
#path = "/data/dataprocessing/interproscan/all_bacilli.tsv"
path = "all_bacilli.tsv"
#path = "bacilli_sample_1000.tsv"
df = pd.read_csv(path, sep = "\t", names = ["Protein_acc", "Sequence_MD5", "Seq_lenght", "Analysis", "Signature_acc", 
                                            "Signature_description", "Start", "Stop", "Score", "Status", 
                                           "Data", "Interpro_acc", "Interpro_descr", "GO_anno", "Pathway_anno"])

In [13]:
df

,Protein_acc,Sequence_MD5,Seq_lenght,Analysis,Signature_acc,Signature_description,Start,Stop,Score,Status,Data,Interpro_acc,Interpro_descr,GO_anno,Pathway_anno
0,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03882,cyclo_dehyd_2: bacteriocin biosynthesis cyclod...,2,131,1.6E-21,T,25-04-2022,IPR022291,"Bacteriocin biosynthesis, cyclodehydratase domain",-,-
1,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03604,TOMM_cyclo_SagD: thiazole/oxazole-forming pept...,161,547,0.0,T,25-04-2022,IPR027624,"Thiazole/oxazole-forming peptide maturase, Sag...",-,-
2,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,ProSiteProfiles,PS51664,YcaO domain profile.,159,547,75.396477,T,25-04-2022,IPR003776,YcaO-like domain,-,-
3,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,Gene3D,G3DSA:3.30.160.660,-,311,452,9.1E-46,T,25-04-2022,-,-,NaN,NaN
4,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,Gene3D,G3DSA:3.30.40.250,-,191,274,9.1E-46,T,25-04-2022,-,-,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200586,gi|530338482|emb|CDG26962.1|,902da1e8e06f749e0ad8bf4986be765f,276,PANTHER,PTHR33745:SF8,BLUE-LIGHT PHOTORECEPTOR,16,275,5.0E-127,T,25-04-2022,-,-,NaN,NaN
4200587,gi|530338482|emb|CDG26962.1|,902da1e8e06f749e0ad8bf4986be765f,276,Pfam,PF01740,STAS domain,166,269,1.1E-6,T,25-04-2022,IPR002645,STAS domain,-,MetaCyc: PWY-5921|Reactome: R-CEL-174362|React...
4200588,gi|530338482|emb|CDG26962.1|,902da1e8e06f749e0ad8bf4986be765f,276,Pfam,PF13426,PAS domain,42,143,2.5E-20,T,25-04-2022,IPR000014,PAS domain,-,MetaCyc: PWY-4702|MetaCyc: PWY-5491|MetaCyc: P...
4200589,gi|530338482|emb|CDG26962.1|,902da1e8e06f749e0ad8bf4986be765f,276,SUPERFAMILY,SSF52091,SpoIIaa-like,163,273,1.18E-12,T,25-04-2022,IPR036513,STAS domain superfamily,-,MetaCyc: PWY-5921|Reactome: R-CEL-174362|React...


In [14]:
df = df[df["Interpro_acc"] != "-"]

In [15]:
df

,Protein_acc,Sequence_MD5,Seq_lenght,Analysis,Signature_acc,Signature_description,Start,Stop,Score,Status,Data,Interpro_acc,Interpro_descr,GO_anno,Pathway_anno
0,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03882,cyclo_dehyd_2: bacteriocin biosynthesis cyclod...,2,131,1.6E-21,T,25-04-2022,IPR022291,"Bacteriocin biosynthesis, cyclodehydratase domain",-,-
1,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03604,TOMM_cyclo_SagD: thiazole/oxazole-forming pept...,161,547,0.0,T,25-04-2022,IPR027624,"Thiazole/oxazole-forming peptide maturase, Sag...",-,-
2,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,ProSiteProfiles,PS51664,YcaO domain profile.,159,547,75.396477,T,25-04-2022,IPR003776,YcaO-like domain,-,-
6,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR00702,TIGR00702: YcaO-type kinase domain,161,501,2.2E-22,T,25-04-2022,IPR003776,YcaO-like domain,-,-
8,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,Pfam,PF02624,"YcaO cyclodehydratase, ATP-ad Mg2+-binding",161,501,8.6E-76,T,25-04-2022,IPR003776,YcaO-like domain,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200584,gi|530338482|emb|CDG26962.1|,902da1e8e06f749e0ad8bf4986be765f,276,SMART,SM00086,pac_2,104,144,2.2E-6,T,25-04-2022,IPR001610,PAC motif,-,Reactome: R-DME-1234158|Reactome: R-DME-123417...
4200585,gi|530338482|emb|CDG26962.1|,902da1e8e06f749e0ad8bf4986be765f,276,CDD,cd00130,PAS,41,141,1.14681E-9,T,25-04-2022,IPR000014,PAS domain,-,MetaCyc: PWY-4702|MetaCyc: PWY-5491|MetaCyc: P...
4200587,gi|530338482|emb|CDG26962.1|,902da1e8e06f749e0ad8bf4986be765f,276,Pfam,PF01740,STAS domain,166,269,1.1E-6,T,25-04-2022,IPR002645,STAS domain,-,MetaCyc: PWY-5921|Reactome: R-CEL-174362|React...
4200588,gi|530338482|emb|CDG26962.1|,902da1e8e06f749e0ad8bf4986be765f,276,Pfam,PF13426,PAS domain,42,143,2.5E-20,T,25-04-2022,IPR000014,PAS domain,-,MetaCyc: PWY-4702|MetaCyc: PWY-5491|MetaCyc: P...


In [16]:
df2 = df[["Protein_acc", "Seq_lenght", "Start", "Stop", "Interpro_acc"]]

In [17]:
df2

,Protein_acc,Seq_lenght,Start,Stop,Interpro_acc
0,gi|29898682|gb|AAP11954.1|,547,2,131,IPR022291
1,gi|29898682|gb|AAP11954.1|,547,161,547,IPR027624
2,gi|29898682|gb|AAP11954.1|,547,159,547,IPR003776
6,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776
8,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776
...,...,...,...,...,...
4200584,gi|530338482|emb|CDG26962.1|,276,104,144,IPR001610
4200585,gi|530338482|emb|CDG26962.1|,276,41,141,IPR000014
4200587,gi|530338482|emb|CDG26962.1|,276,166,269,IPR002645
4200588,gi|530338482|emb|CDG26962.1|,276,42,143,IPR000014


In [22]:
#don't use. does not work and would take very long, use apply()
#for index, row in df2.iterrows():
    #if ((df2["Stop"] - df2["Start"]) / df2["Seq_lenght"]) > 0.90:
        #print("1")
        #df2["Size"] = [1]
    #else:
        #print("0")
        #df2["size"] = [0]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [29]:
def coverage_calc(ddf):
    """
    calculate percentage the feature covers on the total protein length
    use with .apply() method

    returns:
        0 if Interpro_acc < 0.90 (small)
        1 if Interpro_acc > 0.90 (large)
    """
    
    Size = (ddf["Stop"] - ddf["Start"]) / ddf["Seq_lenght"]
    return Size


df2["Size"] = df2.apply(lambda x:coverage_calc(x), axis = 1)

C:\Users\piete\AppData\Local\Temp/ipykernel_27000/1616701365.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Size"] = df2.apply(lambda x:coverage_calc(x), axis = 1)


In [30]:
df2

,Protein_acc,Seq_lenght,Start,Stop,Interpro_acc,Size
0,gi|29898682|gb|AAP11954.1|,547,2,131,IPR022291,0.235832
1,gi|29898682|gb|AAP11954.1|,547,161,547,IPR027624,0.705667
2,gi|29898682|gb|AAP11954.1|,547,159,547,IPR003776,0.709324
6,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776,0.621572
8,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776,0.621572
...,...,...,...,...,...,...
4200584,gi|530338482|emb|CDG26962.1|,276,104,144,IPR001610,0.144928
4200585,gi|530338482|emb|CDG26962.1|,276,41,141,IPR000014,0.362319
4200587,gi|530338482|emb|CDG26962.1|,276,166,269,IPR002645,0.373188
4200588,gi|530338482|emb|CDG26962.1|,276,42,143,IPR000014,0.365942
